In [1]:
from pyspark.sql import SparkSession
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from datetime import datetime
from pyspark.sql.functions import col, trim, regexp_replace

In [ ]:
# Initialize Spark session
spark = SparkSession.builder \
    .appName("Read CSV from HDFS") \
    .config("spark.driver.memory", "8g") \
    .config("spark.hadoop.fs.defaultsFS", "hdfs://localhost:9000") \
    .getOrCreate()

# Read CSV file from HDFS
df = spark.read.format("csv") \
    .option("header", "true") \
    .option("inferSchema", "true") \
    .option("escape", "\"") \
    .load("hdfs://localhost:9000/csv_data/train_data.csv")

df.show(20)

+------+--------------+----------+----------+--------------+-----------+------------------+-----------+-------------+---------------+--------------+-----------+-------+---------------+---------------+------------+--------------------+---------------+
|Row ID|      Order ID|Order Date| Ship Date|     Ship Mode|Customer ID|     Customer Name|    Segment|      Country|           City|         State|Postal Code| Region|     Product ID|       Category|Sub-Category|        Product Name|          Sales|
+------+--------------+----------+----------+--------------+-----------+------------------+-----------+-------------+---------------+--------------+-----------+-------+---------------+---------------+------------+--------------------+---------------+
|     1|CA-2017-152156|08/11/2017|11/11/2017|  Second Class|   CG-12520|       Claire Gute|   Consumer|United States|      Henderson|      Kentucky|      42420|  South|FUR-BO-10001798|      Furniture|   Bookcases|Bush Somerset Col...|         261.

In [3]:
pandas_df = df.toPandas()
print(pandas_df.head())

pandas_df.head(35)

   Row ID        Order ID  Order Date   Ship Date       Ship Mode Customer ID  \
0       1  CA-2017-152156  08/11/2017  11/11/2017    Second Class    CG-12520   
1       2  CA-2017-152156  08/11/2017  11/11/2017    Second Class    CG-12520   
2       3  CA-2017-138688  12/06/2017  16/06/2017    Second Class    DV-13045   
3       4  US-2016-108966  11/10/2016  18/10/2016  Standard Class    SO-20335   
4       5  US-2016-108966  11/10/2016  18/10/2016  Standard Class    SO-20335   

     Customer Name    Segment        Country             City       State  \
0      Claire Gute   Consumer  United States        Henderson    Kentucky   
1      Claire Gute   Consumer  United States        Henderson    Kentucky   
2  Darrin Van Huff  Corporate  United States      Los Angeles  California   
3   Sean O'Donnell   Consumer  United States  Fort Lauderdale     Florida   
4   Sean O'Donnell   Consumer  United States  Fort Lauderdale     Florida   

   Postal Code Region       Product ID         Cat

,Row ID,Order ID,Order Date,Ship Date,Ship Mode,Customer ID,Customer Name,Segment,Country,City,State,Postal Code,Region,Product ID,Category,Sub-Category,Product Name,Sales
0,1,CA-2017-152156,08/11/2017,11/11/2017,Second Class,CG-12520,Claire Gute,Consumer,United States,Henderson,Kentucky,42420.0,South,FUR-BO-10001798,Furniture,Bookcases,Bush Somerset Collection Bookcase,261.96
1,2,CA-2017-152156,08/11/2017,11/11/2017,Second Class,CG-12520,Claire Gute,Consumer,United States,Henderson,Kentucky,42420.0,South,FUR-CH-10000454,Furniture,Chairs,"Hon Deluxe Fabric Upholstered Stacking Chairs,...",731.94
2,3,CA-2017-138688,12/06/2017,16/06/2017,Second Class,DV-13045,Darrin Van Huff,Corporate,United States,Los Angeles,California,90036.0,West,OFF-LA-10000240,Office Supplies,Labels,Self-Adhesive Address Labels for Typewriters b...,14.62
3,4,US-2016-108966,11/10/2016,18/10/2016,Standard Class,SO-20335,Sean O'Donnell,Consumer,United States,Fort Lauderdale,Florida,33311.0,South,FUR-TA-10000577,Furniture,Tables,Bretford CR4500 Series Slim Rectangular Table,957.5775
4,5,US-2016-108966,11/10/2016,18/10/2016,Standard Class,SO-20335,Sean O'Donnell,Consumer,United States,Fort Lauderdale,Florida,33311.0,South,OFF-ST-10000760,Office Supplies,Storage,Eldon Fold 'N Roll Cart System,22.368
5,6,CA-2015-115812,09/06/2015,14/06/2015,Standard Class,BH-11710,Brosina Hoffman,Consumer,United States,Los Angeles,California,90032.0,West,FUR-FU-10001487,Furniture,Furnishings,Eldon Expressions Wood and Plastic Desk Access...,48.86
6,7,CA-2015-115812,09/06/2015,14/06/2015,Standard Class,BH-11710,Brosina Hoffman,Consumer,United States,Los Angeles,California,90032.0,West,OFF-AR-10002833,Office Supplies,Art,Newell 322,7.28
7,8,CA-2015-115812,09/06/2015,14/06/2015,Standard Class,BH-11710,Brosina Hoffman,Consumer,United States,Los Angeles,California,90032.0,West,TEC-PH-10002275,Technology,Phones,Mitel 5320 IP Phone VoIP phone,907.152
8,9,CA-2015-115812,09/06/2015,14/06/2015,Standard Class,BH-11710,Brosina Hoffman,Consumer,United States,Los Angeles,California,90032.0,West,OFF-BI-10003910,Office Supplies,Binders,DXL Angle-View Binders with Locking Rings by S...,18.504
9,10,CA-2015-115812,09/06/2015,14/06/2015,Standard Class,BH-11710,Brosina Hoffman,Consumer,United States,Los Angeles,California,90032.0,West,OFF-AP-10002892,Office Supplies,Appliances,Belkin F5C206VTEL 6 Outlet Surge,114.9


In [4]:
pandas_df.info()

# generate preprocessing.csv from panda_df
pandas_df.to_csv('preprocessing.csv', index=False)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9800 entries, 0 to 9799
Data columns (total 18 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   Row ID         9800 non-null   int32  
 1   Order ID       9800 non-null   object 
 2   Order Date     9800 non-null   object 
 3   Ship Date      9800 non-null   object 
 4   Ship Mode      9800 non-null   object 
 5   Customer ID    9800 non-null   object 
 6   Customer Name  9800 non-null   object 
 7   Segment        9800 non-null   object 
 8   Country        9800 non-null   object 
 9   City           9800 non-null   object 
 10  State          9800 non-null   object 
 11  Postal Code    9789 non-null   float64
 12  Region         9800 non-null   object 
 13  Product ID     9800 non-null   object 
 14  Category       9800 non-null   object 
 15  Sub-Category   9800 non-null   object 
 16  Product Name   9800 non-null   object 
 17  Sales          9800 non-null   object 
dtypes: float

In [5]:
# Convert postal code from XXXXX.0 to XXXXX
pandas_df['Postal Code'] = pandas_df['Postal Code'].apply(lambda x: str(x).split('.')[0] if pd.notnull(x) else x)

print(pandas_df['Postal Code'].head())

# Fill missing values with 0 before converting to integers
pandas_df['Postal Code'] = pandas_df['Postal Code'].fillna(0).astype(int)

print(pandas_df['Postal Code'].head())

0    42420
1    42420
2    90036
3    33311
4    33311
Name: Postal Code, dtype: object
0    42420
1    42420
2    90036
3    33311
4    33311
Name: Postal Code, dtype: int64


In [6]:
pandas_df['Order Date'] = pd.to_datetime(pandas_df['Order Date'], dayfirst=True)
pandas_df['Ship Date'] = pd.to_datetime(pandas_df['Ship Date'], dayfirst=True)

print(pandas_df['Order Date'].head())
print(pandas_df['Ship Date'].head())

0   2017-11-08
1   2017-11-08
2   2017-06-12
3   2016-10-11
4   2016-10-11
Name: Order Date, dtype: datetime64[ns]
0   2017-11-11
1   2017-11-11
2   2017-06-16
3   2016-10-18
4   2016-10-18
Name: Ship Date, dtype: datetime64[ns]


In [7]:
# Remove trailing whitespaces from columns
pandas_df.columns = pandas_df.columns.str.strip()

# Remove non-breaking spaces from columns
pandas_df.columns = pandas_df.columns.str.replace(u'\xa0', u' ')
pandas_df = pandas_df.replace({"\u00A0": " "}, regex=True)

# Generate output1.csv from panda_df
pandas_df.to_csv('output_cleaned_fix.csv', index=False)

pandas_df.head(20)

,Row ID,Order ID,Order Date,Ship Date,Ship Mode,Customer ID,Customer Name,Segment,Country,City,State,Postal Code,Region,Product ID,Category,Sub-Category,Product Name,Sales
0,1,CA-2017-152156,2017-11-08,2017-11-11,Second Class,CG-12520,Claire Gute,Consumer,United States,Henderson,Kentucky,42420,South,FUR-BO-10001798,Furniture,Bookcases,Bush Somerset Collection Bookcase,261.96
1,2,CA-2017-152156,2017-11-08,2017-11-11,Second Class,CG-12520,Claire Gute,Consumer,United States,Henderson,Kentucky,42420,South,FUR-CH-10000454,Furniture,Chairs,"Hon Deluxe Fabric Upholstered Stacking Chairs,...",731.94
2,3,CA-2017-138688,2017-06-12,2017-06-16,Second Class,DV-13045,Darrin Van Huff,Corporate,United States,Los Angeles,California,90036,West,OFF-LA-10000240,Office Supplies,Labels,Self-Adhesive Address Labels for Typewriters b...,14.62
3,4,US-2016-108966,2016-10-11,2016-10-18,Standard Class,SO-20335,Sean O'Donnell,Consumer,United States,Fort Lauderdale,Florida,33311,South,FUR-TA-10000577,Furniture,Tables,Bretford CR4500 Series Slim Rectangular Table,957.5775
4,5,US-2016-108966,2016-10-11,2016-10-18,Standard Class,SO-20335,Sean O'Donnell,Consumer,United States,Fort Lauderdale,Florida,33311,South,OFF-ST-10000760,Office Supplies,Storage,Eldon Fold 'N Roll Cart System,22.368
5,6,CA-2015-115812,2015-06-09,2015-06-14,Standard Class,BH-11710,Brosina Hoffman,Consumer,United States,Los Angeles,California,90032,West,FUR-FU-10001487,Furniture,Furnishings,Eldon Expressions Wood and Plastic Desk Access...,48.86
6,7,CA-2015-115812,2015-06-09,2015-06-14,Standard Class,BH-11710,Brosina Hoffman,Consumer,United States,Los Angeles,California,90032,West,OFF-AR-10002833,Office Supplies,Art,Newell 322,7.28
7,8,CA-2015-115812,2015-06-09,2015-06-14,Standard Class,BH-11710,Brosina Hoffman,Consumer,United States,Los Angeles,California,90032,West,TEC-PH-10002275,Technology,Phones,Mitel 5320 IP Phone VoIP phone,907.152
8,9,CA-2015-115812,2015-06-09,2015-06-14,Standard Class,BH-11710,Brosina Hoffman,Consumer,United States,Los Angeles,California,90032,West,OFF-BI-10003910,Office Supplies,Binders,DXL Angle-View Binders with Locking Rings by S...,18.504
9,10,CA-2015-115812,2015-06-09,2015-06-14,Standard Class,BH-11710,Brosina Hoffman,Consumer,United States,Los Angeles,California,90032,West,OFF-AP-10002892,Office Supplies,Appliances,Belkin F5C206VTEL 6 Outlet Surge,114.9


In [12]:
# Convert to json
pandas_df.to_json('output_cleaned_fix.json', orient='records')

# Check if there is any , in the columns and replace it with ;
pandas_df.columns = pandas_df.columns.str.replace(',', '')
pandas_df = pandas_df.replace({",": ""}, regex=True)

# Generate output2.csv from panda_df
pandas_df.to_csv('output_cleaned_fix2.csv', index=False)

In [13]:
import pandas as pd

# Read the CSV file with proper delimiter and quote handling
out_df = pd.read_csv(
    "C:/Users/YOGA/mdik_dw/datasales/output_cleaned_fix.csv", 
    delimiter=',', 
    quotechar='"', 
    skipinitialspace=True,
    quoting=1 
)

out_df.head(581)


,Row ID,Order ID,Order Date,Ship Date,Ship Mode,Customer ID,Customer Name,Segment,Country,City,State,Postal Code,Region,Product ID,Category,Sub-Category,Product Name,Sales
0,1,CA-2017-152156,2017-11-08,2017-11-11,Second Class,CG-12520,Claire Gute,Consumer,United States,Henderson,Kentucky,42420,South,FUR-BO-10001798,Furniture,Bookcases,Bush Somerset Collection Bookcase,261.96
1,2,CA-2017-152156,2017-11-08,2017-11-11,Second Class,CG-12520,Claire Gute,Consumer,United States,Henderson,Kentucky,42420,South,FUR-CH-10000454,Furniture,Chairs,"Hon Deluxe Fabric Upholstered Stacking Chairs,...",731.94
2,3,CA-2017-138688,2017-06-12,2017-06-16,Second Class,DV-13045,Darrin Van Huff,Corporate,United States,Los Angeles,California,90036,West,OFF-LA-10000240,Office Supplies,Labels,Self-Adhesive Address Labels for Typewriters b...,14.62
3,4,US-2016-108966,2016-10-11,2016-10-18,Standard Class,SO-20335,Sean O'Donnell,Consumer,United States,Fort Lauderdale,Florida,33311,South,FUR-TA-10000577,Furniture,Tables,Bretford CR4500 Series Slim Rectangular Table,957.5775
4,5,US-2016-108966,2016-10-11,2016-10-18,Standard Class,SO-20335,Sean O'Donnell,Consumer,United States,Fort Lauderdale,Florida,33311,South,OFF-ST-10000760,Office Supplies,Storage,Eldon Fold 'N Roll Cart System,22.368
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
576,577,CA-2016-149713,2016-09-18,2016-09-22,Second Class,TG-21640,Trudy Glocke,Consumer,United States,Long Beach,California,90805,West,OFF-PA-10001450,Office Supplies,Paper,Rediform S.O.S. Phone Message Books,19.92
577,578,CA-2016-149713,2016-09-18,2016-09-22,Second Class,TG-21640,Trudy Glocke,Consumer,United States,Long Beach,California,90805,West,OFF-SU-10001574,Office Supplies,Supplies,Acme Value Line Scissors,7.3
578,579,CA-2018-118640,2018-07-20,2018-07-26,Standard Class,CS-11950,Carlos Soltero,Consumer,United States,Chicago,Illinois,60610,Central,OFF-ST-10002974,Office Supplies,Storage,"Trav-L-File Heavy-Duty Shuttle II, Black",69.712
579,580,CA-2018-118640,2018-07-20,2018-07-26,Standard Class,CS-11950,Carlos Soltero,Consumer,United States,Chicago,Illinois,60610,Central,FUR-FU-10001475,Furniture,Furnishings,"Contract Clock, 14""","Brown"""


In [10]:
from sqlalchemy import create_engine

# Konfigurasi koneksi MySQL
user = "root"
password = "admin"
host = "localhost"  # Biasanya 'localhost'
port = "3307"    # Port default MySQL
database = "dbsales"

engine = create_engine(f"mysql+pymysql://{user}:{password}@{host}:{port}/{database}", echo=True)

In [11]:
table_name = "t_product"

out_df.to_sql(
    name=table_name,
    con=engine,
    if_exists="replace",  # Opsi: 'fail', 'replace', atau 'append'
    index=False           # Jangan simpan indeks DataFrame sebagai kolom
)

# Read copy data from out_df to pd_tproduct with column name "Product ID", "Category", "Sub-Category", "Product Name"
pd_tproduct = out_df[["Product ID", "Category", "Sub-Category", "Product Name"]]

table_name = "t_product"
pd_tproduct.to_sql(
    name=table_name,
    con=engine,
    if_exists="replace",
    index=False
)

# Read copy data from out_df to pd_tcustomer with column name "Customer ID", "Customer Name", "Segment"
pd_tcustomer = out_df[["Customer ID", "Customer Name", "Segment"]]

table_name = "t_customer"
pd_tcustomer.to_sql(
    name=table_name,
    con=engine,
    if_exists="replace",
    index=False
)

# Read copy data from out_df to pd_torder with column name "Order ID", "Order Date", "Ship Date", "Ship Mode" 
pd_torder = out_df[["Order ID", "Order Date", "Ship Date", "Ship Mode"]]

table_name = "t_order"
pd_torder.to_sql(
    name=table_name,
    con=engine,
    if_exists="replace",
    index=False
)

# Read copy data from out_df to pd_tlocation with column name "Postal Code", "City", "State", "Region", "Country" 
pd_tlocation = out_df[["Postal Code", "City", "State", "Region", "Country"]]

table_name = "t_location"
pd_tlocation.to_sql(
    name=table_name,
    con=engine,
    if_exists="replace",
    index=False
)

# Read copy data from out_df to pd_tsales with column name "Order ID", "Customer ID", "Product ID", "Sales"
pd_tsales = out_df[["Order ID", "Customer ID", "Product ID", "Sales"]]

table_name = "t_sales"
pd_tsales.to_sql(
    name=table_name,
    con=engine,
    if_exists="replace",
    index=False
)

2024-12-19 21:51:18,521 INFO sqlalchemy.engine.Engine SELECT DATABASE()
2024-12-19 21:51:18,522 INFO sqlalchemy.engine.Engine [raw sql] {}
2024-12-19 21:51:18,526 INFO sqlalchemy.engine.Engine SELECT @@sql_mode
2024-12-19 21:51:18,527 INFO sqlalchemy.engine.Engine [raw sql] {}
2024-12-19 21:51:18,528 INFO sqlalchemy.engine.Engine SELECT @@lower_case_table_names
2024-12-19 21:51:18,531 INFO sqlalchemy.engine.Engine [raw sql] {}
2024-12-19 21:51:18,532 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2024-12-19 21:51:18,538 INFO sqlalchemy.engine.Engine DESCRIBE `dbsales`.`t_product`
2024-12-19 21:51:18,540 INFO sqlalchemy.engine.Engine [raw sql] {}
2024-12-19 21:51:18,545 INFO sqlalchemy.engine.Engine DESCRIBE `dbsales`.`t_product`
2024-12-19 21:51:18,546 INFO sqlalchemy.engine.Engine [raw sql] {}
2024-12-19 21:51:18,548 INFO sqlalchemy.engine.Engine SHOW FULL TABLES FROM `dbsales`
2024-12-19 21:51:18,549 INFO sqlalchemy.engine.Engine [raw sql] {}
2024-12-19 21:51:18,553 INFO sqlalchemy.e

9800

Membuat Delta Lake 